<a href="https://colab.research.google.com/github/joampc/TFM2023/blob/main/noteabooks/Food_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import pandas as pd
import numpy as np
import csv
import re
import json



In [6]:

""" lectura """
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
# Ir a Google Drive , hacer clic derecho sobre la carpeta compartida (Compartida conmigo o Shared with me) y seleccionar Organizar/Añadir acceso directo
# seleccionar en todas las ubicaciones Mi Unidad, y darle Añadir, de esta forma la carpeta TFM aparecera en la lista de carpetas del drive de Google y se podra acceder desde collab.
# COn este comando  lista todo dentro del drive
!ls "/content/drive/MyDrive"


 Alimentos	   'CV Formato Conciso.doc'	    Specs
'Colab Notebooks'  'Gastos Febrero.gsheet'	    TFM
 Colab_Notebooks    HeartDisease_prediction.ipynb


In [7]:
df_alimentos = pd.read_csv('/content/drive/MyDrive/TFM/Alimentos/listado_componentes.csv', delimiter='|', quotechar='"')
df_alimentos.head()

,ID,Nombre,English_Name,Nombre_Cientifico,Codigo_foodex,Parte_Comestible,Descripcion
0,746,Aceite de algodón,Cotton oil,NaN,- () - (),100.0,NaN
1,747,Aceite de cacahuete,Peanut oil,NaN,- () - (),100.0,NaN
2,748,Aceite de coco,Coconut oil,NaN,- () - (),100.0,NaN
3,753,Aceite de colza,Rape oil,NaN,- () - (),100.0,NaN
4,749,Aceite de germen de trigo,Wheat germ oil,NaN,- () - (),100.0,NaN


In [ ]:
#cantidad de alimentos
print(f'Número de filas: {df_alimentos.shape[0]}')


Número de filas: 1052


In [ ]:
df0 = pd.read_csv('/content/drive/MyDrive/TFM/Alimentos/componentes_A.csv' , delimiter='|')
df0.head()

,ID,Grupo,Componente,Valor,Unidad,Fuente
0,746,Proximales,alcohol (etanol),0,g,310
1,746,Proximales,"energía, total",3700 (888),kJ (kcal),236
2,746,Proximales,"grasa, total (lipidos totales)",100,g,30
3,746,Proximales,"proteina, total",0,g,30
4,746,Proximales,agua (humedad),0,g,7


In [8]:
# uniendo todos los datasets de la composicion de alimentos
ruta_base = '/content/drive/MyDrive/TFM/Alimentos/componentes_'

# Lista para almacenar los DataFrames individuales
dfs = []

# Recorrer los archivos correspondientes a cada letra
for letra in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']:
    ruta_archivo = f"{ruta_base}{letra}.csv"
    df = pd.read_csv(ruta_archivo, delimiter='|')
    dfs.append(df)

# Combinar los DataFrames en uno solo
df_composicion = pd.concat(dfs)



# esta es la Información de composición (por 100 g de porción comestible), se repiten varias filas por el mismo ID de alimentos, es probable que esta tabla haya que pivotarla
df_composicion.head()

,ID,Grupo,Componente,Valor,Unidad,Fuente
0,746,Proximales,alcohol (etanol),0,g,310
1,746,Proximales,"energía, total",3700 (888),kJ (kcal),236
2,746,Proximales,"grasa, total (lipidos totales)",100,g,30
3,746,Proximales,"proteina, total",0,g,30
4,746,Proximales,agua (humedad),0,g,7


In [9]:
num_filas = df_composicion.shape[0]
print("Número de filas:", num_filas)

Número de filas: 39012


In [ ]:
# revisando todo lo que pertenece a un componente
df_746 = df_composicion[df_composicion['ID'] == 746]
print(df_746)

In [ ]:
# revisando las unidades presentes en las columna unidad
df_composicion['Unidad'].unique()

array(['g', 'kJ (kcal)', '-', 'mg', 'ug'], dtype=object)

In [ ]:
#tipos de las columnas
print(df_composicion.dtypes)


ID            object
Grupo         object
Componente    object
Valor         object
Unidad        object
Fuente        object
dtype: object


In [ ]:
# contando cuantos elementos tienen en el valor traza
print(df_composicion[df_composicion['Valor'] == 'traza']['Valor'].value_counts())

traza    1468
Name: Valor, dtype: int64


In [ ]:
# ver los valores no numericos que estan en la columna valor y q no corresponde al valor de la energia que tiene parentesis
valores_numericos = pd.to_numeric(df_composicion['Valor'], errors='coerce')
valores_no_numericos = df_composicion.loc[(valores_numericos.isna()) & (df_composicion['Unidad'] != 'kJ (kcal)'), 'Valor'].unique()

print(valores_no_numericos)

['-' 'traza' nan]


In [ ]:
# ver registros que tiene valor NAN en valor, revisando el site, son aquellos que no tienen nada en esa celda
registros_nan = df_composicion[df_composicion['Valor'].isna()]
registros_nan.head()


,ID,Grupo,Componente,Valor,Unidad,Fuente
1277,2473,Minerales,ioduro,NaN,ug,258
1477,2364,Minerales,ioduro,NaN,ug,38
1885,2183,Hidratos de Carbono,"fibra, dietetica total",NaN,g,30
1903,2183,Vitaminas,Viamina E equivalentes de alfa tocoferol de ac...,NaN,mg,30
1917,2183,Minerales,ioduro,NaN,ug,30


In [11]:
# vamos a considerar que aquellos alimentos que en el valor de un componente tienen un valor de traza, aunque es muy pequeño, puede ser considerado peligroso en caso de las alergias,
# inicialmente le voy a poner -1, cuando tenga todos los valores en mg le asignare un valor menor que el menor valor de la columna valor en mg
# las medidas estan en gramos,miligramos y microgramos, vamos a unificar todo a miligramos
def convert_to_mg(value, unit):
    if pd.isna(value) or value == '-':
        return value
    elif value == 'traza':
        return -1
    elif unit == 'g':
        return float(value) * 1000
    elif unit == 'ug':
        return float(value) / 1000
    elif unit == 'kJ (kcal)':
        values = re.findall(r'\d+', value)
        return {
            'energia_Kcal': values[1],
            'energia_KJ': values[0]
        }
    else:
        return value
df_composicion['Valor'] = df_composicion.apply(lambda row: convert_to_mg(row['Valor'], row['Unidad']), axis=1)
df_composicion.head(10)

,ID,Grupo,Componente,Valor,Unidad,Fuente
0,746,Proximales,alcohol (etanol),0.0,g,310
1,746,Proximales,"energía, total","{'energia_Kcal': '888', 'energia_KJ': '3700'}",kJ (kcal),236
2,746,Proximales,"grasa, total (lipidos totales)",100000.0,g,30
3,746,Proximales,"proteina, total",0.0,g,30
4,746,Proximales,agua (humedad),0.0,g,7
5,746,Hidratos de Carbono,"fibra, dietetica total",0.0,g,30
6,746,Hidratos de Carbono,carbohidratos,0.0,g,30
7,746,Grasas,ácido graso 22:6 n-3 (ácido docosahexaenóico),-,-,-
8,746,Grasas,"ácidos grasos, monoinsaturados totales",17800.0,g,30
9,746,Grasas,"ácidos grasos, poliinsaturados totales",51900.0,g,30


In [ ]:
# buscar el minimo valor mayor que 0, esto nos servira para imputar los que digan traza por un valor muy peque;o pero no 0 teniendo en cuenta los alergicos
def find_min_value(df):
    filtered_df = df.loc[(df['Valor'] != '-') & (df['Valor'].notna()) & (df['Valor'] != -1) & (df['Unidad'] != 'kJ (kcal)'), ['ID', 'Valor', 'Componente']]
    filtered_df['Valor'] = pd.to_numeric(filtered_df['Valor'], errors='coerce')
    filtered_df = filtered_df[filtered_df['Valor'] > 0]
    if not filtered_df.empty:
        min_value = filtered_df['Valor'].min()
        min_value_row = filtered_df.loc[filtered_df['Valor'] == min_value, ['ID', 'Valor', 'Componente']].iloc[0]
        return min_value_row
    else:
        return None
min_value_row = find_min_value(df_composicion)
if min_value_row is not None:
    print('ID:', min_value_row['ID'])
    print('Valor mínimo:', min_value_row['Valor'])
    print('Componente:', min_value_row['Componente'])

ID: 790
Valor mínimo: 1.6999999999999998e-06
Componente: Vitamina A equivalentes de retinol de actividades de retinos y carotenoides


In [12]:
# Filtrar registros duplicados
duplicates = df_composicion[df_composicion.duplicated(subset=['ID', 'Componente'], keep=False)]

# Mostrar registros duplicados
len(duplicates)

1600

In [ ]:
#Buscando un alimento con error en las composiciones
# Filtrar el DataFrame para encontrar una fila con 'acido' en la columna 'Componente' y sin 'Grasas' en la columna 'Grupo'
filtro = (df_composicion['Componente'].str.lower().str.startswith('ácido', na=False)) & (df_composicion['Grupo'] != 'Grasas')
fila = df_composicion.loc[filtro]
# Mostrar la fila resultante
print(fila)

       ID       Grupo                                  Componente  Valor  \
800  2237  Proximales      ácidos grasos, monoinsaturados totales  0.009   
801  2237  Proximales      ácidos grasos, poliinsaturados totales    0.1   
802  2237  Proximales             ácidos grasos saturados totales  0.043   
803  2237  Proximales                  ácido graso 12:0 (láurico)      -   
804  2237  Proximales          ácido graso 14:0 (ácido mirístico)      -   
805  2237  Proximales          ácido graso 16:0 (ácido palmítico)      -   
806  2237  Proximales          ácido graso 18:0 (ácido esteárico)      -   
807  2237  Proximales     ácido graso 18:1 n-9 cis (ácido oléico)      -   
809  2237  Proximales                            ácido graso 18:2      -   
810  2237  Proximales                            ácido graso 18:3      -   
811  2237  Proximales   ácido graso 20:4 n-6 (ácido araquidónico)      -   
812  2237  Proximales  ácido graso 20:5 (ácido eicosapentaenóico)      -   

    Unidad 

In [ ]:
filas_2237 = df_composicion.loc[df_composicion['ID'] == 2237]

# Muestra las filas resultantes
print(filas_2237)
#el elemento 2237 tiene una estructura erronea en la base de datos teniendo en cuenta que es un elemento que no aportara mucho (Nispero conserva en su jugo) y que muchos de sus valores nutricionales
# estan vacio vamos a eliminarlo de los datasets

       ID       Grupo                                         Componente  \
800  2237  Proximales             ácidos grasos, monoinsaturados totales   
801  2237  Proximales             ácidos grasos, poliinsaturados totales   
802  2237  Proximales                    ácidos grasos saturados totales   
803  2237  Proximales                         ácido graso 12:0 (láurico)   
804  2237  Proximales                 ácido graso 14:0 (ácido mirístico)   
805  2237  Proximales                 ácido graso 16:0 (ácido palmítico)   
806  2237  Proximales                 ácido graso 18:0 (ácido esteárico)   
807  2237  Proximales            ácido graso 18:1 n-9 cis (ácido oléico)   
808  2237  Proximales                                         colesterol   
809  2237  Proximales                                   ácido graso 18:2   
810  2237  Proximales                                   ácido graso 18:3   
811  2237  Proximales          ácido graso 20:4 n-6 (ácido araquidónico)   
812  2237  P

In [14]:
#elimino el alimento con Id 2237 de los dataframes
df_alimentos = df_alimentos.drop(df_alimentos[df_alimentos['ID'] == 2237].index)
df_composicion = df_composicion.drop(df_composicion[df_composicion['ID'] == 2237].index)

In [25]:
#eliminando valores duplicados
df_unique = df_composicion.drop_duplicates(subset=['ID', 'Componente'])

# Copiar el DataFrame antes de realizar la asignación
df_unique = df_unique.copy()

# definir prefijos correspondientes a cada grupo para agregarlo al nombre de columna asi se sabra a que grupo pertence cuando se pivotee la tabla
prefixes = {
    'Proximales': 'pro_',
    'Hidratos de Carbono': 'hidC_',
    'Grasas': 'gra_',
    'Vitaminas': 'vit_',
    'Minerales': 'min_'
}

# Modificar nombres de columnas y agregar prefijos según el valor de 'Grupo'
df_unique['Componente'] = df_unique.apply(lambda row: f"{prefixes.get(row['Grupo'], '')}{row['Componente']}", axis=1)


# Pivotar el DataFrame
df_pivot = df_unique.pivot(index='ID', columns='Componente', values='Valor')


# Modificar nombres de columnas
df_pivot.columns = df_pivot.columns.str.replace(' ', '_').str.replace(',', '_').str.replace('(', '').str.replace(')', '')
df_pivot.columns = df_pivot.columns.map(lambda x: re.sub('_+', '_', x))


# Separar columna 'energía, total' en 'energia_Kcal' y 'energia_KJ'
energia_values = df_pivot['pro_energía_total'].apply(lambda x: x if isinstance(x, dict) else {})

df_pivot['pro_energia_Kcal'] = energia_values.apply(lambda x: x.get('energia_Kcal'))
df_pivot['pro_energia_KJ'] = energia_values.apply(lambda x: x.get('energia_KJ'))

# Eliminar columnas 'energía_total' del DataFrame resultante
df_pivot.drop(['pro_energía_total'], axis=1, inplace=True)

# Mostrar DataFrame resultante
df_pivot

<ipython-input-25-983ff6f6314e>:21: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_pivot = df_unique.pivot(index='ID', columns='Componente', values='Valor')
<ipython-input-25-983ff6f6314e>:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_pivot.columns = df_pivot.columns.str.replace(' ', '_').str.replace(',', '_').str.replace('(', '').str.replace(')', '')
<ipython-input-25-983ff6f6314e>:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_pivot.columns = df_pivot.columns.str.replace(' ', '_').str.replace(',', '_').str.replace('(', '').str.replace(')', '')


Componente,gra_colesterol,gra_ácido_graso_12:0_láurico,gra_ácido_graso_14:0_ácido_mirístico,gra_ácido_graso_16:0_ácido_palmítico,gra_ácido_graso_18:0_ácido_esteárico,gra_ácido_graso_18:1_n-9_cis_ácido_oléico,gra_ácido_graso_18:2,gra_ácido_graso_18:3,gra_ácido_graso_20:4_n-6_ácido_araquidónico,gra_ácido_graso_20:5_ácido_eicosapentaenóico,...,vit_Vitamina_B-12,vit_Vitamina_B-6_Total,vit_Vitamina_C_ácido_ascórbico,vit_Vitamina_D,vit_equivalentes_de_niacina_totales,vit_folato_total,vit_riboflavina,vit_tiamina,pro_energia_Kcal,pro_energia_KJ
ID,,,,,,,,,,,,,,,,,,,,,
680,80,-,-,-,-,-,-,-,-,-,...,0.001,0.46,-1,0.0006,7,0.002,0.16,0.9,292,1216
681,8,-,-,-,-,-,-,-,-,-,...,0.0001,0.01,0.5,0.00001,0.7,0.00166,0.08,0.02,38,158
682,100,-,-,-,-,-,-,-,-,-,...,0.0008,0.3,0,0.00001,1.8,0.002,0.11,0.36,546,2277
683,0,-,-,-,-,-,-,-,-,-,...,0.00036,0.15,4.2,0.0003,1.3,0.0211,0.18,0.1,48,201
685,3.4,-,-,-,-,-,-,-,-,-,...,0.0002,0.15,4.2,0.0003,1.3,0.0211,0.13,0.1,112,466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2700,78,20.0,272.0,4436.0,2195.0,8265.0,2051.0,153.0,-,-,...,0.0001,0.4,-,-,5.2,-1,0.34,0.35,254,1058
2701,74,27.0,124.0,1952.0,992.0,3527.0,760.0,47.0,-,-,...,0.0002,0.39,-,-,5.1,-1,0.36,0.64,165,687
2711,88.56,30.0,450.0,3180.0,2060.0,5490.0,580.0,50.0,70.0,10.0,...,0.00236,0.54,-,0.02,3.07,0.0019,0.05,0.1,265,1103


In [21]:
df_pivot.columns


Index(['ID', 'gra_colesterol', 'gra_ácido_graso_12:0_láurico',
       'gra_ácido_graso_14:0_ácido_mirístico',
       'gra_ácido_graso_16:0_ácido_palmítico',
       'gra_ácido_graso_18:0_ácido_esteárico',
       'gra_ácido_graso_18:1_n-9_cis_ácido_oléico', 'gra_ácido_graso_18:2',
       'gra_ácido_graso_18:3', 'gra_ácido_graso_20:4_n-6_ácido_araquidónico',
       'gra_ácido_graso_20:5_ácido_eicosapentaenóico',
       'gra_ácido_graso_22:6_n-3_ácido_docosahexaenóico',
       'gra_ácidos_grasos_saturados_totales',
       'gra_ácidos_grasos_monoinsaturados_totales',
       'gra_ácidos_grasos_poliinsaturados_totales', 'hidC_carbohidratos',
       'hidC_fibra_dietetica_total', 'min_calcio', 'min_fósforo',
       'min_hierro_total', 'min_ioduro', 'min_magnesio', 'min_potasio',
       'min_selenio_total', 'min_sodio', 'min_zinc_cinc', 'pro_agua_humedad',
       'pro_alcohol_etanol', 'pro_grasa_total_lipidos_totales',
       'pro_proteina_total',
       'vit_Viamina_E_equivalentes_de_alfa_tocof

In [26]:
# Restablecer el índice del dataframe df_pivot
df_pivot_idx = df_pivot.reset_index()

# Unir los dataframes utilizando el método join()
df_alimentos_composicion = df_alimentos.join(df_pivot_idx.set_index('ID'), on='ID')
df_alimentos_composicion.drop(['Nombre_Cientifico', 'Codigo_foodex', 'Descripcion'], axis=1, inplace=True)


# Imprimir el resultado
print(df_alimentos_composicion)

        ID                     Nombre           English_Name  \
0      746          Aceite de algodón             Cotton oil   
1      747        Aceite de cacahuete             Peanut oil   
2      748             Aceite de coco            Coconut oil   
3      753            Aceite de colza               Rape oil   
4      749  Aceite de germen de trigo         Wheat germ oil   
...    ...                        ...                    ...   
1047  1079   Zumo de pomelo, envasado       Grapefruit juice   
1048  2248     Zumo de tomate, fresco    Tomato, fresh juice   
1049  2365                Zumo de uva            Grape juice   
1050   791    Zumo de uva y melocotón  Grape and peach juice   
1051  2252  Zumo de zanahoria, fresco    Carrot, fresh juice   

      Parte_Comestible gra_colesterol gra_ácido_graso_12:0_láurico  \
0                100.0              0                            -   
1                100.0              0                            -   
2                100.

In [ ]:

#el fichero grupo de alimentos puede servir para agrupar los ingredientes
df_grupos = pd.read_csv('/content/drive/MyDrive/TFM/Alimentos/grupos.csv', delimiter='|')
df_grupos.head()

,ID Select,Texto,ID
0,1,Lácteos y derivados,2479
1,1,Lácteos y derivados,2480
2,1,Lácteos y derivados,2481
3,1,Lácteos y derivados,2482
4,1,Lácteos y derivados,2519


In [ ]:
grupos =df_grupos['Texto'].unique().tolist()
for grupo in grupos:
  print(grupo)

Lácteos y derivados
Huevos y derivados
Cárnicos y derivados
Pescados, moluscos, reptiles, crustáceos y derivados
Grasas y aceites
Cereales y derivados
Legumbres, semillas, frutos secos y derivados
Verduras, hortalizas y derivados
Frutas y derivados
Azúcar, chocolate y derivados
Bebidas (no lácteas)
Miscelánea


In [ ]:
#esto es un dataset informativo que describe lo que significa cada elemento de la composicion de los alimentos
df_desc_componente = pd.read_csv('/content/drive/MyDrive/TFM/Alimentos/descripcion_componentes.csv', delimiter='|')
df_desc_componente.shape[0]
df_desc_componente.head(40)

,componente,Descripcion
0,alcohol (etanol),Componente de las bebidas alcohólicas obtenida...
1,"energía, total",La energía expresa las calorías que un determi...
2,"grasa, total (lipidos totales)",La grasa es un macronutriente que aporta gran ...
3,"proteina, total",La proteína es un macronutriente cuya principa...
4,agua (humedad),El contenido de agua de un alimento determina ...
5,"fibra, dietetica total",La fibra dietética es un componente de los ali...
6,carbohidratos,Este componente es uno de los macronutrientes ...
7,ácido graso 22:6 n-3 (ácido docosahexaenóico),NaN
8,"ácidos grasos, monoinsaturados totales",Son componentes de las grasas de los alimentos...
9,"ácidos grasos, poliinsaturados totales",Son componentes de las grasas que se encuentra...
